# Comparison of Embedding Methods

### This notebook demonstrates the calculation of Adjusted Rand Scores on a variety of directed graphs. We will compare the performance of three spectral embedding methods: Adjacency Spectral Embedding (ASE), Laplacian Spectral Embedding (LSE), and Scikit-Learn's Spectral Embedding.

In [18]:
import time
import warnings
warnings.filterwarnings("ignore")
import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import adjusted_rand_score
from sklearn.manifold import spectral_embedding
from sklearn.mixture import GaussianMixture

from graspologic.embed import AdjacencySpectralEmbed, LaplacianSpectralEmbed

### The following function generates a k-neighbors graph using latent positions derived from a block matrix. It then applies a specified spectral embedding method, followed by Gaussian Mixture Clustering to compute the Adjusted Rand Score. This process is repeated over multiple iterations, with each score being recorded.

In [19]:
def calc_ari(kn_graph, n_sims, embed_method, labels_true, cov_type):
    rows = []

    for _ in range(n_sims):

        #choose embedding method
        if embed_method == "ase":
            ase = AdjacencySpectralEmbed(n_components=2)
            Xhat, Yhat = ase.fit_transform(kn_graph)
        elif embed_method == "lse":
            lse = LaplacianSpectralEmbed(n_components=2)
            Xhat, Yhat = lse.fit_transform(kn_graph)
        elif embed_method == "sklearn":
            Xhat = spectral_embedding(kn_graph, n_components=2)

        #concatenate Xhat and Yhat if using ase or lse
        if embed_method == "ase" or embed_method == "lse":
            Xhat = np.concatenate((Xhat, Yhat), axis=1)

        #calculate the ARI score
        gm_ic = GaussianMixture(n_components=2, covariance_type=cov_type)
        labels_mclust = gm_ic.fit_predict(Xhat)
        ari = adjusted_rand_score(labels_true, labels_mclust)

        #save the embedding method used with the corresponding ARI value
        result = {
            "test": embed_method,
            "ari": ari
        }
        rows.append(result)


    results = pd.DataFrame(rows)
    return results

### Now we present 2 cases, in which LSE performs the best in case 1, while ASE performs the best in case 2. In each case, we choose a certain number of vertices and block matrix, and run the above method using all three embedding methods. We then calculate the means of the ARI scores for each embedding method across the number of iterations to determine which embedding method gave the best results.

In this simulation, we consider two block matrices, one being an affinity matrix, and the other being a core periphery matrix, as detailed in the [Two Truths paper](https://www.pnas.org/doi/10.1073/pnas.1814462116). The structure of each matrix is discussed below:

Let

$$
B_{\text{affinity}} = \begin{bmatrix} a & b \\ b & c \end{bmatrix}, \quad \text{where } a, c \gg b
$$

and 

$$
B_{\text{core}} = \begin{bmatrix} a & b \\ b & c \end{bmatrix}, \quad \text{where } a \gg b, c \text{ or } c \gg b, a
$$

In [20]:
#case 1: LSE does best
n_verts_lse = 100
n_sims = 50
B_core = np.array([[0.011, 0.027], [0.027, 0.079]])
labels_lse = int(n_verts_lse/2) * [0] + int(n_verts_lse/2) * [1]

#generate kn graph
#make probability matrix from block matrix
P = np.zeros((n_verts_lse, n_verts_lse))
P[0:int(n_verts_lse/2),0:int(n_verts_lse/2)] = B_core[0, 0]
P[int(n_verts_lse/2):n_verts_lse, int(n_verts_lse/2):n_verts_lse] = B_core[1, 1]
P[0:int(n_verts_lse/2), int(n_verts_lse/2):n_verts_lse] = B_core[0, 1]
P[int(n_verts_lse/2):n_verts_lse, 0:int(n_verts_lse/2)] = B_core[1, 0]

#make latent position matrix
U, S, V = np.linalg.svd(P)

#sample half the points from U
X = U[0:int(n_verts_lse/2), 0:2] @ np.sqrt(np.diag(S[0:2]))

#sample half the points from V^T
Y = V.T[int(n_verts_lse/2):n_verts_lse, 0:2] @ np.sqrt(np.diag(S[0:2]))

#concatenate the two matrices to get the full latent position matrix
lat_mat = np.concatenate((X, Y), axis=0)

#get k_neighbors graph from latent position matrix (k=sqrt(n))
kn_graph = kneighbors_graph(lat_mat, n_neighbors=int(np.sqrt(n_verts_lse)))
kn_graph = kn_graph.toarray()

ari_aff_lse_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "lse", labels_true = labels_lse, cov_type = "full")
ari_aff_ase_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "ase", labels_true = labels_lse, cov_type = "full")
ari_aff_sklearn_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "sklearn", labels_true = labels_lse, cov_type = "full")
ari_aff_df = pd.concat([ari_aff_lse_df, ari_aff_ase_df, ari_aff_sklearn_df])

#groupby the means for each embedding method across the simulations
ari_aff_means = ari_aff_df.groupby(["test"]).mean()
ari_aff_means

,ari
test,
ase,0.653139
lse,0.808924
sklearn,0.311481


In [21]:
#case 2: ASE does best
n_verts_ase = 400
n_sims = 50
B_affinity = np.array([[0.050, 0.013], [0.013, 0.051]])
labels_ase = int(n_verts_ase/2) * [0] + int(n_verts_ase/2) * [1]

#generate kn graph
#make probability matrix from block matrix
P = np.zeros((n_verts_ase, n_verts_ase))
P[0:int(n_verts_ase/2),0:int(n_verts_ase/2)] = B_affinity[0, 0]
P[int(n_verts_ase/2):n_verts_ase, int(n_verts_ase/2):n_verts_ase] = B_affinity[1, 1]
P[0:int(n_verts_ase/2), int(n_verts_ase/2):n_verts_ase] = B_affinity[0, 1]
P[int(n_verts_ase/2):n_verts_ase, 0:int(n_verts_ase/2)] = B_affinity[1, 0]

#make latent position matrix
U, S, V = np.linalg.svd(P)

#sample half the points from U
X = U[0:int(n_verts_ase/2), 0:2] @ np.sqrt(np.diag(S[0:2]))

#sample half the points from V^T
Y = V.T[int(n_verts_ase/2):n_verts_ase, 0:2] @ np.sqrt(np.diag(S[0:2]))

#concatenate the two matrices to get the full latent position matrix
lat_mat = np.concatenate((X, Y), axis=0)

#get k_neighbors graph from latent position matrix (k=sqrt(n))
kn_graph = kneighbors_graph(lat_mat, n_neighbors=int(np.sqrt(n_verts_ase)))
kn_graph = kn_graph.toarray()

ari_core_ase_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "ase", labels_true = labels_ase, cov_type = "full")
ari_core_lse_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "lse", labels_true = labels_ase, cov_type = "full")
ari_core_sklearn_df = calc_ari(kn_graph = kn_graph, n_sims = n_sims, embed_method = "sklearn", labels_true = labels_ase, cov_type = "full")
ari_core_df = pd.concat([ari_core_ase_df, ari_core_lse_df, ari_core_sklearn_df])
ari_core_means = ari_core_df.groupby(["test"]).mean()
ari_core_means

,ari
test,
ase,0.656647
lse,0.524430
sklearn,0.241342
